In [1]:
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.cm as cmx

In [2]:
clusters = pd.read_json("./files/cluster.json")

ValueError: Expected object or value

In [ ]:
def plotSteps(data):
    for runIdx in range(0, len(data)):
        fig, ax = plt.subplots()
        iteration = data.iloc[runIdx]["iteration"]
        clusters = data.iloc[runIdx]["clusters"]
        norm = colors.Normalize(vmin=0, vmax=len(clusters))
        print("iteration: %s, clusters: %s" % (iteration, len(clusters)))
        for cIdx in range(0, len(clusters)):
            rgba_color = cm.gnuplot(norm(cIdx))
            pointsX = [point[0] for point in clusters[cIdx]["points"]]
            pointsY = [point[1] for point in clusters[cIdx]["points"]]
            ax.scatter(pointsX, pointsY, color=rgba_color)
            ax.scatter(clusters[cIdx]["centroid"][0], clusters[cIdx]["centroid"][1], color=rgba_color, marker="+")

        plt.show()

In [ ]:
plotSteps(clusters)

In [3]:
def plot(data):
    for runIdx in range(0, len(data)):
        fig, ax = plt.subplots()
        clusters_step1 = data.iloc[runIdx]["steps"][0]["clusters"]
        clusters_step2 = data.iloc[runIdx]["steps"][1]["clusters"]
        norm = colors.Normalize(vmin=0, vmax=len(clusters_step1))
        for cIdx in range(0, len(clusters_step1)):
            rgba_color = cm.gnuplot(norm(cIdx))
            ax.plot(clusters_step1[cIdx]["syntheticCenter"], "--", color=rgba_color, label="cluster %s" % cIdx)
            ax.plot(clusters_step2[cIdx]["syntheticCenter"], color=rgba_color, label="cluster %s - rescheduled" % cIdx)
            print(clusters_step2[cIdx]["metric"])
        ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)

        ax.set(xlabel='Hours', ylabel='Power (kW)',title='')
        plt.show()

In [4]:
def plot_aggm(summary, file_name=None): 
    fig, ax = plt.subplots()
    rgba_color = cm.gnuplot(norm(0))
    ax.plot(range(1, len(summary["s1. agg m"]) + 1), summary["s1. agg m"], "--", color=rgba_color, label="s1. agg m")
    ax.plot(range(1, len(summary["s2. agg m"]) + 1), summary["s2. agg m"], color=rgba_color, label="s2. agg m")
    lgd = ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)
    ax.set(xlabel='K clusters', ylabel='Aggregate PAR',title='')
    plt.show()
    if file_name is not None:
        fig.savefig(file_name, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return fig
    
def plot_maxm(summary, file_name=None): 
    fig, ax = plt.subplots()
    rgba_color = cm.gnuplot(norm(0))
    ax.plot(range(1, len(summary["s1. agg m"]) + 1), summary["s1. max m"], "--", color=rgba_color, label="s1. max m")
    ax.plot(range(1, len(summary["s2. agg m"]) + 1), summary["s2. max m"], color=rgba_color, label="s2. max m")
    lgd = ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)
    ax.set(xlabel='K clusters', ylabel='Max. Aggregate PAR',title='')
    plt.show()
    if file_name is not None:
        fig.savefig(file_name, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return fig
    
def plot_peak(summary, file_name=None):
    fig, ax = plt.subplots()
    rgba_color = cm.gnuplot(norm(0))
    ax.plot(range(1, len(summary["s1. agg m"]) + 1), summary["s1. peak"], "--", color=rgba_color, label="s1. peak")
    ax.plot(range(1, len(summary["s2. agg m"]) + 1), summary["s2. peak"], color=rgba_color, label="s2. peak")
    lgd = ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)
    ax.set(xlabel='K clusters', ylabel='Peak power (kW)',title='')
    plt.show()
    if file_name is not None:
        fig.savefig(file_name, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return fig

# PAR Aggregate

In [107]:
summaryBatch_global = pd.read_json("./files/summaryBatch.json")
#summaryBatch_global = summaryBatch_global[["k","clusters","s1. agg m","s1. max m","s1. peak","s2. agg m","s2. max m","s2. peak","total m"]]
#batch_global = pd.read_json("./files/batch.json")

k = 2
sampleSizeMin = 0.8
sampleSizeMax = 1.0
steps = 1


def sampleSizeRange():
    return (x/10 for x in range(int(sampleSizeMin * 10), int((sampleSizeMax * 10) + 1)))

In [108]:
index_list = []
step_list = []

for i in range(0 , len(summaryBatch_global)):
    register = summaryBatch_global.iloc[i]
    
    crossfold = json_normalize(register["crossfold"])
    splits = crossfold["splits"].values[0]
    steps = summaryBatch_global.iloc[i]["step"]
    for j in range(0, splits):
        step = json_normalize(steps[j])
        for ki in range(1, k+1):
            index_list.append((float("{0:.2f}".format(crossfold["sampleSize"].values[0])), j, ki))
            step_list.append(step.iloc[ki-1])
        
    
json_normalize(summaryBatch_global.iloc[0]["crossfold"])
json_normalize(summaryBatch_global.iloc[0]["step"][0])

index = pd.MultiIndex.from_tuples(index_list, names=["sampleSize", "split", "k"])
summaryBatch_global_treated = pd.DataFrame(step_list, index=index)
summaryBatch_global_treated

clusters  k  s1. agg m  s1. max m    s1. peak  s2. agg m  \
sampleSize split k                                                             
1.0        0     1      [50]  1   1.451786   1.451786  171.170633   1.451786   
                 2  [25, 25]  2   1.136210   1.843565  105.326017   1.136210   

                    s2. max m    s2. peak   total m  
sampleSize split k                                   
1.0        0     1   1.451786  171.170633  1.451786  
                 2   1.843565  105.326017  1.451786

In [102]:
k2 = 2
query = f"k == {k2} and sampleSize == {sampleSizeMin}"
summaryBatch_global_treated.query(query)["s1. agg m"].mean()
summaryBatch_global_treated.query(query)["s1. max m"].mean()
summaryBatch_global_treated.query(query)["s1. peak"].std()

nan

In [103]:
with open('summaryBatch.tex','w') as tf:
    tf.write(summaryBatch_global.to_latex())

In [104]:
index_list = []
step_list = []

for j in range(1, k+1):
    for ssize in sampleSizeRange():
        
        index_list.append((ssize, "mean", j))
        query = f"k == {j} and sampleSize == {ssize}"
        data = {"s1. agg m" : summaryBatch_global_treated.query(query)["s1. agg m"].mean(),
                "s1. max m" : summaryBatch_global_treated.query(query)["s1. max m"].mean(), 
                "s1. peak" : summaryBatch_global_treated.query(query)["s1. peak"].mean()}
        index = ["s1. agg m","s1. max m","s1. peak"]
        if (steps >= 2):
            data.update({"s2. agg m" : summaryBatch_global_treated.query(query)["s2. agg m"].mean(),
                "s2. max m" : summaryBatch_global_treated.query(query)["s2. max m"].mean(), 
                "s2. peak" : summaryBatch_global_treated.query(query)["s2. peak"].mean()})
            index = index + ["s2. agg m","s2. max m","s2. peak"]
            
        s = pd.Series(data,index=index) 
        step_list.append(s)
        #print(f"sampleSize = {ssize}, k = {j}")
        
        index_list.append((ssize, "stdv", j))
        query = f"k == {j} and sampleSize == {ssize}"
        data = {"s1. agg m" : summaryBatch_global_treated.query(query)["s1. agg m"].std(),
                "s1. max m" : summaryBatch_global_treated.query(query)["s1. max m"].std(), 
                "s1. peak" : summaryBatch_global_treated.query(query)["s1. peak"].std()}
        index = ["s1. agg m","s1. max m","s1. peak"]
        if (steps >= 2):
            data.update({"s2. agg m" : summaryBatch_global_treated.query(query)["s2. agg m"].std(),
                "s2. max m" : summaryBatch_global_treated.query(query)["s2. max m"].std(), 
                "s2. peak" : summaryBatch_global_treated.query(query)["s2. peak"].std()})
            index = index + ["s2. agg m","s2. max m","s2. peak"]
            
        s = pd.Series(data,index=index) 
        step_list.append(s)

        
index = pd.MultiIndex.from_tuples(index_list, names=["sampleSize", "metric", "k"])
summaryBatch_global_statistics = pd.DataFrame(step_list, index=index)
summaryBatch_global_statistics

TypeError: '>=' not supported between instances of 'list' and 'int'

In [79]:
def getMetrics(metric, summaryMetric):

    index_list = [sampleSizeRange()]
    cols = [x for x in range(1, k+1)]
    step_list = []

    for ssize in sampleSizeRange():
        query = f"sampleSize == {ssize} and metric == \"{summaryMetric}\" "
        dataAllK = summaryBatch_global_statistics.query(query)[f"{metric}"]
        dictAllK = {}
        for j in range(1, k+1):
            d = dataAllK.reset_index().query(f"k == {j}")[f"{metric}"]
            dictAllK[f"{j}"] = d.values
        s = pd.Series(dictAllK, name=f"{ssize}")
        step_list.append(s)

    return pd.DataFrame(step_list)
    
getMetrics("s1. agg m", "mean")
getMetrics("s1. agg m", "stdv")

,1,2,3
0.8,[0.095904899031],[0.0748629667593],[0.0625413514146]
0.9,[0.061754706342],[0.0584759944394],[0.0608020632314]
1.0,[0.0],[0.0387179838613],[0.109939285615]


In [51]:
series = [pd.Series(np.random.rand(3), name=c) for c in list('abcdefg')]
series
pd.DataFrame.from_items([(s.name, s) for s in series])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  This is separate from the ipykernel package so we can avoid doing imports until


,a,b,c,d,e,f,g
0,0.160780,0.312639,0.737680,0.495611,0.765506,0.827995,0.063007
1,0.523206,0.113421,0.479314,0.969279,0.025397,0.627076,0.023188
2,0.393152,0.257559,0.556803,0.020505,0.596032,0.200862,0.471754
